# Launch of Deepethogram - Infer

## 1. Copy movies
This script will repare all videos (corrupted for one reason I ignore), and will then add them using deepethogram fonction (it calculate std and mean whhile adding the videos)

In [ ]:
import os
import subprocess
from deepethogram import projects
import re

def repair_video_with_ffmpeg(video_path, output_dir):
    import shlex

    filename = os.path.basename(video_path)
    name_no_ext = os.path.splitext(filename)[0]
    safe_name = name_no_ext.replace(" ", "")
    repaired_filename = f"{safe_name}.mp4"
    repaired_path = os.path.join(output_dir, repaired_filename)

    if os.path.exists(repaired_path):
        print(f"[✓] Already repaired: {repaired_filename}")
        return repaired_path

    print(f"[⚙] Repairing: {filename}")
    try:
        cmd = f"""ffmpeg -y -err_detect ignore_err -i "{video_path}" \
        -vf "fps=30" -c:v libx264 -preset veryfast -crf 24 \
        -c:a aac -strict experimental "{repaired_path}" """
        
        subprocess.run(shlex.split(cmd), check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return repaired_path
    except subprocess.CalledProcessError as e:
        print(f"[✗] Error repairing {filename}: {e}")
        return None


def ajouter_tous_videos_au_projet(project_config_path, root_video_dir, mode='copy', extensions=None):
    if extensions is None:
        extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv']

    project = projects.load_config(project_config_path)
    execution_dir = os.getcwd()

    videos_paths = []
    pattern = re.compile(r".*_(\d{2})-(\d{2})-(\d{2})") 
    for root, dirs, files in os.walk(root_video_dir):
        for file in files:
            match = pattern.match(file)
            if match and "CONT" in file and not file.startswith("CONT"):
                hour = int(match.group(1))
                if 8 > hour or hour >= 20:
                    if os.path.splitext(file)[1].lower() in extensions:
                        full_path = os.path.join(root, file)
                        videos_paths.append(full_path)

    print(f"Found {len(videos_paths)} videos to check and add.")
    cont = 0
    for video_path in videos_paths:
        video_name = os.path.basename(video_path)

        repaired_path = repair_video_with_ffmpeg(video_path, output_dir=execution_dir)
        if repaired_path:
            cont += 1
            try:
                new_path = projects.add_video_to_project(project, repaired_path, mode=mode)
                print(f"[+] Added: {repaired_path} → {new_path} . {cont} / {len(videos_paths)}")
                os.remove(repaired_path)
                print(f"[🗑] Deleted temporary file: {repaired_path}")
            except Exception as e:
                print(f"[✗] Error adding to project {repaired_path}: {e}")
                os.remove(repaired_path)
                print(f"[🗑] Deleted temporary file: {repaired_path}")
        else:
            print(f"[!] Skipped: {video_path} (repair failed)")

# === CONFIGURACIÓN ===
config_path = r"D:\LBN\Maternal_auto_classification_TS7_deepethogram\project_config.yaml"
videos_dir = r"Z:\Marion\2. Tests - Manuels - Data\3. LBN\3. Rawdata\1. VEAVE_LBN-CONT"

ajouter_tous_videos_au_projet(config_path, videos_dir, mode='copy')

## 2. Infer videos
We'll then pass our model through the new data to predict the behaviours found in each frame

In [ ]:
import subprocess

# Comando para activar el entorno y ejecutar el script en Windows
cmd = 'conda activate deg_env && python d:\\LBN\\tryInfer.py'

# Ejecutar el comando en shell
subprocess.run(cmd, shell=True, check=True)

## 3. Transform output
We'll finally transform the .h5 output file on a .csv file

In [2]:
from deepethogram.postprocessing import get_postprocessor_from_cfg
import os
import h5py
import pandas as pd
from deepethogram import projects

def convertir_outputs_a_csv_con_postprocess(data_dir, cfg):
    config = projects.load_config(cfg)
    output_dir = r"D:\LBN\Maternal_behaviour_results_cont"
    os.makedirs(output_dir, exist_ok=True)

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('outputs.h5'):
                outputs_path = os.path.join(root, file)
                try:
                    with h5py.File(outputs_path, 'r') as f:
                        group = f['resnet18']
                        probabilities = group['P'][:]
                        thresholds = group['thresholds'][:]
                        class_names = [x.decode('utf-8') for x in group['class_names'][:]]

                    postprocessor = get_postprocessor_from_cfg(config, thresholds)
                    estimated_labels = postprocessor(probabilities)
                    df = pd.DataFrame(estimated_labels, columns=class_names)

                    # Nouveau nom de fichier avec même base mais dans le dossier output
                    base_name = os.path.splitext(os.path.basename(outputs_path))[0]
                    prediction_fname = os.path.join(output_dir, base_name + '_labels_predictions.csv')
                    df.to_csv(prediction_fname, index=False)
                    print(f"[✓] Guardado postprocesado: {prediction_fname}")

                except Exception as e:
                    print(f"[✗] Error con {outputs_path}: {e}")

# Appel
directorio_data = r"D:\LBN\Maternal_auto_classification_TS7_deepethogram\DATA"
cfg = r"D:\LBN\Maternal_auto_classification_TS7_deepethogram\project_config.yaml"
convertir_outputs_a_csv_con_postprocess(directorio_data, cfg)


c:\Users\TeamGranon\anaconda3\envs\deg_solo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[WARNING] Ajustando bout_lengths automáticamente: thresholds=10, bout_lengths=None
[✓] Guardado postprocesado: D:\LBN\Maternal_behaviour_results_cont\Cage1-CONT-2024-12-08_20-08-34_outputs_labels_predictions.csv
[WARNING] Ajustando bout_lengths automáticamente: thresholds=10, bout_lengths=None
[✓] Guardado postprocesado: D:\LBN\Maternal_behaviour_results_cont\Cage1-CONT-2024-12-08_20-18-34_outputs_labels_predictions.csv
[WARNING] Ajustando bout_lengths automáticamente: thresholds=10, bout_lengths=None
[✓] Guardado postprocesado: D:\LBN\Maternal_behaviour_results_cont\Cage1-CONT-2024-12-08_20-28-34_outputs_labels_predictions.csv
[WARNING] Ajustando bout_lengths automáticamente: thresholds=10, bout_lengths=None
[✓] Guardado postprocesado: D:\LBN\Maternal_behaviour_results_cont\Cage1-CONT-2024-12-08_20-38-34_outputs_labels_predictions.csv
[WARNING] Ajustando bout_lengths automáticamente: thresholds=10, bout_lengths=None
[✓] Guardado postprocesado: D:\LBN\Maternal_behaviour_results_cont\Ca